# **Tutorial** - (semi)-supervised topic modeling
(last updated 11-09-2022)

In this tutorial, we will be looking at a new feature of BERTopic, namely (semi)-supervised topic modeling! This allows us to steer the dimensionality reduction of the embeddings into a space that closely follows any labels you might already have.

## Semi-supervised modeling
(semi)-supervised topic modeling is a class of methods that allows the user to perform topic modeling with previously defined labels. This might help nudge the model towards specific topics or classes for which you have labels.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts that each is assigned to one of 20 topics:

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
targets = data["target"]
target_names = data["target_names"]
classes = [data["target_names"][i] for i in data["target"]]

Each document can be put into one of the following categories:

In [ ]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# **(semi)-Supervised modeling**


## Basic Model
Before we start with semi-supervised modeling, let us first take a look at the output of the basic model.

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs)

2024-03-17 21:45:36,610 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2024-03-17 21:46:39,523 - BERTopic - Embedding - Completed ✓
2024-03-17 21:46:39,525 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 21:47:18,533 - BERTopic - Dimensionality - Completed ✓
2024-03-17 21:47:18,535 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-17 21:47:23,132 - BERTopic - Cluster - Completed ✓
2024-03-17 21:47:23,159 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-17 21:47:29,056 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3690,-1_the_of_to_and,"[the, of, to, and, in, that, you, is, it, for]",[THE WHITE HOUSE\n\n Office o...
1,0,917,0_space_launch_nasa_orbit,"[space, launch, nasa, orbit, shuttle, spacecra...",[Archive-name: space/addresses\nLast-modified:...
2,1,910,1_window_server_file_motif,"[window, server, file, motif, widget, entry, p...","[Archive-name: Xt-FAQ\nVersion: $Id: FAQ-Xt,v ..."
3,2,910,2_he_game_year_baseball,"[he, game, year, baseball, team, games, player...","[\n\tOh, yeah. Dave Winfield--marginal player..."
4,3,900,3_car_cars_engine_ford,"[car, cars, engine, ford, it, my, the, oil, de...","[Hi,\nI need your help with a problem I have w..."
5,4,885,4_key_encryption_clipper_chip,"[key, encryption, clipper, chip, keys, privacy...",[-----BEGIN PGP SIGNED MESSAGE-----\n\nPlease ...
6,5,861,5_mac_apple_drive_monitor,"[mac, apple, drive, monitor, with, disk, mhz, ...",[Here is how I modified my Quadra 700 for high...
7,6,858,6_gun_guns_firearms_they,"[gun, guns, firearms, they, the, fbi, of, weap...","[\n Mr. Parsli, I have to take exception..."
8,7,809,7_image_jpeg_images_graphics,"[image, jpeg, images, graphics, format, gif, f...",[Archive-name: graphics/faq\n\nThis message is...
9,8,541,8_windows_dos_file_files,"[windows, dos, file, files, os2, microsoft, 31...",[This is the official Request for Discussion (...


The topics that were created mostly make sense. There are some clearly defined topics such as "nasa, orbit, spacecraft, moon" but also some topics that seem mostly derived from other topics. We can visualize this by extracting the topic representations per class and see if our unsupervised model closely resembles this.

**NOTE**: You can **hover** over the bars to see the representation per class!!

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_unsupervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
fig_unsupervised

20it [00:07,  2.71it/s]


The results do seem promising. Topics like "nasa, space, etc" seem to be clearly related to sci.space, but some topics were created that span many categories. For example, we expect the topic "bike, bikes, etc"  to only appear in rec.motorcycles.  

## Semi-supervised
In the example above you might notice that some topics were somewhat smushed together. What we would like to see is a clear separation between those topics. Fortunately, we have to labels and can use them to improve the model.

Since we are not interested in any other topics, this method is called semi-supervised topic modeling. In practice, this means that we have the labels of some documents but not all.

For this example let's say we only have the labels of all computer-related categories:

In [ ]:
 labels_to_add = ['comp.graphics', 'comp.os.ms-windows.misc',
                  'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
                  'comp.windows.x',]
 indices = [target_names.index(label) for label in labels_to_add]
 new_labels = [label if label in indices else -1 for label in targets]

When generating our new labels it is important to mark unknown classes as **-1**. Next, we use those newly constructed labels to again run BERTopic:

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=new_labels)

2024-03-17 21:47:40,026 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2024-03-17 21:48:30,639 - BERTopic - Embedding - Completed ✓
2024-03-17 21:48:30,640 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 21:48:51,664 - BERTopic - Dimensionality - Completed ✓
2024-03-17 21:48:51,666 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-17 21:48:52,551 - BERTopic - Cluster - Completed ✓
2024-03-17 21:48:52,562 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-17 21:48:58,021 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5858,-1_the_of_to_and,"[the, of, to, and, is, in, for, it, that, you]","[Tony-\n\nI read your post, it was nothing new..."
1,0,1835,0_game_team_he_games,"[game, team, he, games, players, season, hocke...",[NHL RESULTS FOR GAMES PLAYED 4/15/93.\n\n----...
2,1,878,1_window_server_motif_widget,"[window, server, motif, widget, xterm, display...",[Archive-name: x-faq/part3\nLast-modified: 199...
3,2,842,2_image_jpeg_images_gif,"[image, jpeg, images, gif, format, file, files...",[I am happy to announce the first public relea...
4,3,577,3_key_clipper_chip_encryption,"[key, clipper, chip, encryption, keys, escrow,...","[Hmm, followup on my own posting... Well, who ..."
5,4,526,4_ites_cheek_yep_dir,"[ites, cheek, yep, dir, huh, ken, art, why, ea...","[dir\n, \n \n ..."
6,5,445,5_you_your_post_jim,"[you, your, post, jim, context, that, not, me,...",[I apologize if this article is slightly confu...
7,6,439,6_israel_israeli_jews_arab,"[israel, israeli, jews, arab, arabs, jewish, p...",[From: Center for Policy Research <cpr>\nSubje...
8,7,288,7_bike_riding_ride_my,"[bike, riding, ride, my, you, lane, car, road,...",[\nI'll tell you my story as an example of wha...
9,8,188,8_car_cars_mustang_ford,"[car, cars, mustang, ford, engine, v8, convert...",[\nWhile I don't read normally read this group...


Finally, we can again extract the topics per class to see if our semi-supervised approach had some effect:

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_semi_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)
fig_semi_supervised

20it [00:07,  2.81it/s]


We can clearly see that many more topics about computers were created and that the seperation between those topics are solid. This indicates that even if you do not have all the labels, you can definitely improve the model!

However, there are still some clusters that could be improved with the labels that we have.

## Supervised

Finally, we are going to be using all labels. These labels help BERTopic understand where most clusters can be found. However, this does not mean that it will only find the 20 clusters that we have defined. If there are sub-clusters to be found, then there is a good chance BERTopic will find them!

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=targets)

2024-03-17 21:49:07,460 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2024-03-17 21:49:57,751 - BERTopic - Embedding - Completed ✓
2024-03-17 21:49:57,752 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 21:50:19,103 - BERTopic - Dimensionality - Completed ✓
2024-03-17 21:50:19,106 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-17 21:50:19,848 - BERTopic - Cluster - Completed ✓
2024-03-17 21:50:19,857 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-17 21:50:25,528 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3690,-1_the_of_to_and,"[the, of, to, and, in, that, you, is, it, for]",[THE WHITE HOUSE\n\n Office o...
1,0,917,0_space_launch_nasa_orbit,"[space, launch, nasa, orbit, shuttle, spacecra...",[Archive-name: space/addresses\nLast-modified:...
2,1,910,1_window_server_file_motif,"[window, server, file, motif, widget, entry, p...","[Archive-name: Xt-FAQ\nVersion: $Id: FAQ-Xt,v ..."
3,2,910,2_he_game_year_baseball,"[he, game, year, baseball, team, games, player...","[\n\tOh, yeah. Dave Winfield--marginal player..."
4,3,900,3_car_cars_engine_ford,"[car, cars, engine, ford, it, my, the, oil, de...","[Hi,\nI need your help with a problem I have w..."
5,4,885,4_key_encryption_clipper_chip,"[key, encryption, clipper, chip, keys, privacy...",[-----BEGIN PGP SIGNED MESSAGE-----\n\nPlease ...
6,5,861,5_mac_apple_drive_monitor,"[mac, apple, drive, monitor, with, disk, mhz, ...",[Here is how I modified my Quadra 700 for high...
7,6,858,6_gun_guns_firearms_they,"[gun, guns, firearms, they, the, fbi, of, weap...","[\n Mr. Parsli, I have to take exception..."
8,7,809,7_image_jpeg_images_graphics,"[image, jpeg, images, graphics, format, gif, f...",[Archive-name: graphics/faq\n\nThis message is...
9,8,541,8_windows_dos_file_files,"[windows, dos, file, files, os2, microsoft, 31...",[This is the official Request for Discussion (...


Not only do we see a nice seperation of the topics, there are significantly less outliers which shows that BERTopic has improved in connecting the documents to topics.

Let's see the results by again visualizing the topic representation per class:

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)
fig_supervised

20it [00:06,  3.05it/s]


Now that we have used all labels, BERTopic seems to closely match our pre-defined labels. Moreover, it still allows to discover topics that were not previously defined. Thus, you can use this method to find unknown topics in pre-defined topics!